# Keyword Search

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
%%capture
!pip install cohere
!pip install "weaviate-client==3.*"
!pip install python-dotenv

In [3]:
import os
import getpass
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [4]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [5]:
_set_env("WEAVIATE_API_KEY")
_set_env("COHERE_API_KEY")
os.environ['WEAVIATE_API_URL'] = "https://cohere-demo.weaviate.network/"

WEAVIATE_API_KEY: ··········
COHERE_API_KEY: ··········


Let's start by imporing Weaviate to access the Wikipedia database.

In [6]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(
    api_key=os.environ['WEAVIATE_API_KEY'])



In [7]:
client = weaviate.Client(
    url=os.environ['WEAVIATE_API_URL'],
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],
    }
)

In [8]:
client.is_ready()

True

# Keyword Search

In [9]:
def keyword_search(query,
                   results_lang='en',
                   properties = ["title","url","text"],
                   num_results=3):

    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }

    response = (
        client.query.get("Articles", properties)
        .with_bm25(
            query=query
        )
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
        )

    result = response['data']['Get']['Articles']
    return result

In [10]:
query = "What is the most viewed televised event?"
keyword_search_results = keyword_search(query)
print(keyword_search_results)

[{'text': 'The most active Gamergate supporters or "Gamergaters" said that Gamergate was a movement for ethics in games journalism, for protecting the "gamer" identity, and for opposing "political correctness" in video games and that any harassment of women was done by others not affiliated with Gamergate. They argued that the close relationships between journalists and developers demonstrated a conspiracy among reviewers to focus on progressive social issues. Some supporters pointed to what they considered disproportionate praise for games such as "Depression Quest" and "Gone Home", which feature unconventional gameplay and stories with social implications, while they viewed traditional AAA games as downplayed. False claims of the "ethics in game journalism" had started as early as 2012, when Geoff Keighley was accused of such unethical behavior when he was presenting information about "Halo 4" among advertisements for Mountain Dew and Doritos, an event called "Doritosgate" in the gam

In [11]:
keyword_search_results

[{'text': 'The most active Gamergate supporters or "Gamergaters" said that Gamergate was a movement for ethics in games journalism, for protecting the "gamer" identity, and for opposing "political correctness" in video games and that any harassment of women was done by others not affiliated with Gamergate. They argued that the close relationships between journalists and developers demonstrated a conspiracy among reviewers to focus on progressive social issues. Some supporters pointed to what they considered disproportionate praise for games such as "Depression Quest" and "Gone Home", which feature unconventional gameplay and stories with social implications, while they viewed traditional AAA games as downplayed. False claims of the "ethics in game journalism" had started as early as 2012, when Geoff Keighley was accused of such unethical behavior when he was presenting information about "Halo 4" among advertisements for Mountain Dew and Doritos, an event called "Doritosgate" in the gam

### Try modifying the search options
- Other languages to try: `en, de, fr, es, it, ja, ar, zh, ko, hi`

In [12]:
properties = ["text", "title", "url",
             "views", "lang"]

In [13]:
def print_result(result):
    """ Print results with colorful formatting """
    for i,item in enumerate(result):
        print(f'item {i}')
        for key in item.keys():
            print(f"{key}:{item.get(key)}")
            print()
        print()

In [14]:
print_result(keyword_search_results)

item 0
text:The most active Gamergate supporters or "Gamergaters" said that Gamergate was a movement for ethics in games journalism, for protecting the "gamer" identity, and for opposing "political correctness" in video games and that any harassment of women was done by others not affiliated with Gamergate. They argued that the close relationships between journalists and developers demonstrated a conspiracy among reviewers to focus on progressive social issues. Some supporters pointed to what they considered disproportionate praise for games such as "Depression Quest" and "Gone Home", which feature unconventional gameplay and stories with social implications, while they viewed traditional AAA games as downplayed. False claims of the "ethics in game journalism" had started as early as 2012, when Geoff Keighley was accused of such unethical behavior when he was presenting information about "Halo 4" among advertisements for Mountain Dew and Doritos, an event called "Doritosgate" in the ga

In [15]:
query = "What color is the sky?"
keyword_search_results = keyword_search(query, results_lang='en')
print_result(keyword_search_results)

item 0
text:The failure of "No Man's Sky" promotional aspects has affected other space simulation and open world games that are based on the premise of providing a vast ranging sandbox for players, as players have become wary of the broad claims that these games might make. Novaquark, the developers of the upcoming open-world "Dual Universe", found themselves struggling to complete their Kickstarter funding in the months immediately after "No Man's Sky" release, but have recognised the need to be open and transparent to potential funders on what the game will and will not have. Fenix Fire, the developers of the space exploration game "", used the various question-and-answers that Sean Murray had to handle during the pre-release period to gauge what players were looking for in such games and guide development of their own game. According to a report from "Kotaku", BioWare had envisioned that "" would use procedural generation for creating a universe to explore prior to "No Man's Sky" an

In [16]:
keyword_search_results

[{'text': 'The failure of "No Man\'s Sky" promotional aspects has affected other space simulation and open world games that are based on the premise of providing a vast ranging sandbox for players, as players have become wary of the broad claims that these games might make. Novaquark, the developers of the upcoming open-world "Dual Universe", found themselves struggling to complete their Kickstarter funding in the months immediately after "No Man\'s Sky" release, but have recognised the need to be open and transparent to potential funders on what the game will and will not have. Fenix Fire, the developers of the space exploration game "", used the various question-and-answers that Sean Murray had to handle during the pre-release period to gauge what players were looking for in such games and guide development of their own game. According to a report from "Kotaku", BioWare had envisioned that "" would use procedural generation for creating a universe to explore prior to "No Man\'s Sky" 

## How to get your own API key

For this course, an API key is provided for you.  If you would like to develop projects with Cohere's API outside of this classroom, you can register for an API key [here](https://dashboard.cohere.ai/welcome/register?utm_source=partner&utm_medium=website&utm_campaign=DeeplearningAI).